<a target="_blank" href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/2-Advance/FinRL_Ensemble_StockTrading_ICAIF_2020.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# # ## install finrl library
# !pip install wrds
# !pip install swig
# # !pip install -q condacolab
# # import condacolab
# # condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


In [2]:
# !pip install yfinance pandas numpy matplotlib stockstats gym stable-baselines3 tensorflow pyfolio

In [3]:
# !pip install pandas_market_calendars

<a id='1.3'></a>
## 2.3. Import Packages

In [4]:
import warnings
warnings.filterwarnings("ignore")

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime
import statistics 
from stable_baselines3.common.callbacks import BaseCallback

%matplotlib inline
from FinRL.finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from FinRL.finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from FinRL.finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from FinRL.finrl.agents.stablebaselines3.models import DRLAgent,DRLEnsembleAgent
from FinRL.finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from stable_baselines3.common.vec_env import DummyVecEnv
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

In [16]:
df_account_value = pd.read_csv('df_account_value.csv', index_col=0)
df_account_value_a2c = pd.read_csv('df_account_value_a2c.csv', index_col=0)
df_account_value_ppo = pd.read_csv('df_account_value_ppo.csv', index_col=0)

In [20]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)

==============Get Backtest Results===========
Annual return          0.005958
Cumulative returns     0.025567
Annual volatility      0.101266
Sharpe ratio           0.109669
Calmar ratio           0.026887
Stability              0.694287
Max drawdown          -0.221592
Omega ratio            1.022288
Sortino ratio          0.140565
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.086881
Daily value at risk   -0.012714
dtype: float64


In [21]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value_ppo)
perf_stats_all = pd.DataFrame(perf_stats_all)
# ==============Get Backtest Results===========
# Annual return               0.379725
# Cumulative returns          0.083797
# Annual volatility      338567.947564
# Sharpe ratio                2.017211
# Calmar ratio                0.379725
# Stability                   0.324152
# Max drawdown               -1.000000
# Omega ratio             22751.286315
# Sortino ratio          155782.345252
# Skew                             NaN
# Kurtosis                         NaN
# Tail ratio                  9.187028
# Daily value at risk    -39945.381418

==============Get Backtest Results===========
Annual return          0.081611
Cumulative returns     0.405763
Annual volatility      0.151941
Sharpe ratio           0.593186
Calmar ratio           0.374945
Stability              0.720479
Max drawdown          -0.217661
Omega ratio            1.120370
Sortino ratio          0.805448
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.979209
Daily value at risk   -0.018785
dtype: float64


In [22]:
#baseline stats
print("==============Get Baseline Stats===========")
df_dji_ = get_baseline(
        ticker="^DJI",
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(df_dji_, value_col_name = 'close')

[*********************100%***********************]  1 of 1 completed

==============Get Baseline Stats===========
Shape of DataFrame:  (1070, 8)
Annual return          0.044268
Cumulative returns     0.201921
Annual volatility      0.194809
Sharpe ratio           0.320864
Calmar ratio           0.119365
Stability              0.838783
Max drawdown          -0.370862
Omega ratio            1.076707
Sortino ratio          0.428540
Skew                        NaN
Kurtosis                    NaN
Tail ratio             0.874737
Daily value at risk   -0.024296
dtype: float64


In [23]:
df_dji = pd.DataFrame()
df_dji['date'] = df_account_value['date']
df_dji['dji'] = df_dji_['close'] / df_dji_['close'][0] * 1000000
print("df_dji: ", df_dji)
df_dji.to_csv("df_dji.csv")
df_dji = df_dji.set_index(df_dji.columns[0])
print("df_dji: ", df_dji)
df_dji.to_csv("df_dji+.csv")

df_account_value.to_csv('df_account_value.csv')


df_dji:              date           dji
0     2015-12-31  1.000000e+06
1     2016-01-04  9.841556e+05
2     2016-01-05  9.847134e+05
3     2016-01-06  9.702428e+05
4     2016-01-07  9.477229e+05
...          ...           ...
1066  2020-03-27  1.241707e+06
1067  2020-03-30  1.281345e+06
1068  2020-03-31  1.257798e+06
1069  2020-04-01  1.201921e+06
1070  2020-04-02           NaN

[1071 rows x 2 columns]
df_dji:                       dji
date                    
2015-12-31  1.000000e+06
2016-01-04  9.841556e+05
2016-01-05  9.847134e+05
2016-01-06  9.702428e+05
2016-01-07  9.477229e+05
...                  ...
2020-03-27  1.241707e+06
2020-03-30  1.281345e+06
2020-03-31  1.257798e+06
2020-04-01  1.201921e+06
2020-04-02           NaN

[1071 rows x 1 columns]


<a id='6.2'></a>
## 7.2 BackTestPlot

In [32]:
import numpy as np
import pandas as pd
import cvxpy as cp
processed = pd.read_csv('processed.csv', index_col=0)
stock_dimension = len(processed.tic.unique())
df_result_ensemble = pd.DataFrame({'date': df_account_value['date'], 
                                 'ensemble': df_account_value['account_value']}).set_index('date')

df_result_a2c = pd.DataFrame({'date': df_account_value_a2c['date'], 
                             'a2c': df_account_value_a2c['account_value']}).set_index('date')

df_result_ppo = pd.DataFrame({'date': df_account_value_ppo['date'], 
                             'ppo': df_account_value_ppo['account_value']}).set_index('date')

# Merge all DataFrames using their date index
result = df_result_ensemble.join([df_result_ppo, df_result_a2c, df_dji])

TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2015-09-30'
TEST_START_DATE = '2015-09-30'
TEST_END_DATE = '2020-05-08'
# Helper function to process dataframe for portfolio optimization
def process_df_for_mvo(df):
    df = df.sort_values(['date', 'tic'], ignore_index=True)[['date', 'tic', 'close']]
    fst = df.iloc[0:stock_dimension, :]
    tic = fst['tic'].tolist()

    mvo = pd.DataFrame()
    for k in range(len(tic)):
        mvo[tic[k]] = 0

    for i in range(df.shape[0] // stock_dimension):
        n = df.iloc[i * stock_dimension : (i + 1) * stock_dimension, :]
        date = n['date'].iloc[0]  # Use .iloc to ensure single date
        mvo.loc[date] = n['close'].tolist()

    return mvo

# Helper function to compute stock returns (in decimal form)
def StockReturnsComputing(StockPrice, Rows, Columns):
    StockReturn = np.zeros([Rows - 1, Columns])
    for j in range(Columns):
        for i in range(Rows - 1):
            StockReturn[i, j] = (StockPrice[i + 1, j] - StockPrice[i, j]) / StockPrice[i, j]
    return StockReturn

# Assuming data_split, processed_full, TRAIN_START_DATE, TRAIN_END_DATE, TEST_START_DATE, TEST_END_DATE, and stock_dimension are defined
train_mvo = data_split(processed, TRAIN_START_DATE, TRAIN_END_DATE).reset_index()
trade_mvo = data_split(processed, TEST_START_DATE, TEST_END_DATE).reset_index()

# Process data for training and trading periods
StockData = process_df_for_mvo(train_mvo)
TradeData = process_df_for_mvo(trade_mvo)

# Get tickers
tickers = StockData.columns.tolist()
print("StockData tickers:", tickers)
print("Number of tickers:", len(tickers))

# Compute asset returns
arStockPrices = np.asarray(StockData)
Rows, Cols = arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)
print("arReturns shape:", arReturns.shape)

# Compute mean returns and covariance matrix
meanReturns = np.mean(arReturns, axis=0)
covReturns = np.cov(arReturns, rowvar=False)
print("covReturns shape:", covReturns.shape)

# Check for NaN in covariance matrix
if np.any(np.isnan(covReturns)):
    print("Warning: covReturns contains NaN values. Check input data for missing or invalid prices.")
    raise ValueError("NaN values in covariance matrix")

# Convert meanReturns and covReturns to pandas with tickers
meanReturns = pd.Series(meanReturns, index=tickers)
covReturns = pd.DataFrame(covReturns, index=tickers, columns=tickers)

# ── Replace pypfopt EfficientFrontier with CVXPY-based Min Variance ─────────────
# 1) Extract Σ (covariance matrix) in the correct ticker order
Σ = covReturns.loc[tickers, tickers].to_numpy()  # shape (n, n)
n = len(tickers)

# 2) Define CVXPY variable
w = cp.Variable(n)

# 3) Define objective: minimize w^T Σ w
objective = cp.Minimize(cp.quad_form(w, Σ))

# 4) Define constraints: sum(w) == 1, 0 <= w_i <= 0.5
constraints = [
    cp.sum(w) == 1,
    w >= 0,
    w <= 0.5
]

# 5) Formulate and solve the problem
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.SCS)

# 6) Check solver status
if prob.status not in ["optimal", "optimal_inaccurate"]:
    raise ValueError(f"CVXPY did not find an optimal solution (status={prob.status})")

# 7) Extract raw weights and clean them (clamp negatives to zero, re-normalize)
raw_weights = w.value
cleaned = np.maximum(raw_weights, 0)
cleaned = cleaned / cleaned.sum()

# 8) Map cleaned weights back into a dict keyed by ticker
cleaned_weights_min_var = {tickers[i]: float(cleaned[i]) for i in range(n)}

# Debug: Check weights
print("Cleaned weights keys:", list(cleaned_weights_min_var.keys()))
print("Number of weights:", len(cleaned_weights_min_var))

# Verify ticker alignment
missing_tickers = [tic for tic in tickers if tic not in cleaned_weights_min_var]
if missing_tickers:
    print("Error: Missing tickers in weights:", missing_tickers)
    raise KeyError(f"Tickers {missing_tickers} not found in cleaned_weights_min_var")

# Compute dollar allocations ($1,000,000 total portfolio value)
min_var_weights = np.array([1_000_000 * cleaned_weights_min_var[tic] for tic in tickers])

# Compute number of shares based on last training period prices
# (assuming fractional shares are allowed for backtesting)
LastPrice = np.array([1 / p for p in StockData.tail(1).to_numpy()[0]])
Initial_Portfolio_min_var = np.multiply(min_var_weights, LastPrice)

# Calculate portfolio value over the trade period
# Ensure TradeData columns align with `tickers` order
TradeData = TradeData[tickers]
Portfolio_Assets_min_var = TradeData @ Initial_Portfolio_min_var

# Create DataFrame for Min Variance results
df_account_value_min_var = pd.DataFrame({
    'date': TradeData.index,
    'account_value': Portfolio_Assets_min_var
}).reset_index(drop=True)

# Prepare result DataFrame for comparison
df_result_min_var = pd.DataFrame({
    'date': df_account_value_min_var['date'],
    'min_var': df_account_value_min_var['account_value']
}).set_index('date')

# Assuming df_result_ensemble, df_result_ppo, df_result_a2c, and df_dji are defined
# Merge all strategy results
result = df_result_ensemble.join([df_result_ppo, df_result_a2c, df_result_min_var, df_dji], how='inner')

# Calculate cumulative returns
initial_ensemble = result['ensemble'].iloc[0]
initial_ppo = result['ppo'].iloc[0]
initial_a2c = result['a2c'].iloc[0]
initial_min_var = result['min_var'].iloc[0]
initial_dji = result['dji'].iloc[0]

df_cumulative = pd.DataFrame({
    'Ensemble': (result['ensemble'] / initial_ensemble - 1) * 100,
    'PPO':      (result['ppo']      / initial_ppo      - 1) * 100,
    'A2C':      (result['a2c']      / initial_a2c      - 1) * 100,
    'Min Var':  (result['min_var']  / initial_min_var  - 1) * 100,
    'DJI':      (result['dji']      / initial_dji      - 1) * 100
}, index=result.index)


StockData tickers: ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WMT', 'XOM']
Number of tickers: 29
arReturns shape: (1696, 29)
covReturns shape: (29, 29)
Cleaned weights keys: ['AAPL', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DD', 'DIS', 'GE', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WMT', 'XOM']
Number of weights: 29


In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# … all your data-prep up through df_cumulative …
# ensure the index is parsed as real timestamps
df_cumulative.index = pd.to_datetime(df_cumulative.index)

# Create a figure and axes explicitly
fig, ax = plt.subplots(figsize=(15, 5))

# Plot the cumulative returns (this draws your four lines exactly as before)
df_cumulative.plot(ax=ax)

# —— now override the default x-axis ticks & grid —— 

# 1. Major ticks every quarter (Jan, Apr, Jul, Oct)
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=[1,4,7,10]))

# 2. Format each tick as “Jan 4” on line 1, “2016” on line 2
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d\n%Y'))

# 3. Draw vertical gridlines at those quarter-points, plus horizontal at y-majors
ax.grid(axis='x', which='major', linestyle='--', linewidth=0.5)
ax.grid(axis='y', which='major', linestyle='--', linewidth=0.5)

# 4. Tweak tick label alignment
plt.setp(ax.get_xticklabels(), rotation=0, ha='center')

# —— finish off your labels/legend/title exactly as before —— 

ax.set_title('Cumulative Return Comparison of Trading Strategies vith Transaction Cost')
ax.set_xlabel('Date')
ax.set_ylabel('Cumulative Return (%)')
ax.legend(['Ensemble', 'PPO', 'A2C', 'DJIA', 'Min var'])
plt.tight_layout()
plt.show()


In [30]:
# assuming `processed` has columns ['date','tic',...,'turbulence']
turb_per_date = (
    processed[['date','turbulence']]
    .drop_duplicates('date')
    .set_index('date')
    .sort_index()
)

# now turb_per_date is one row per date:
print(turb_per_date.sample(20))

            turbulence
date                  
2015-07-30   61.872816
2018-04-12   28.966606
2009-12-01    0.000000
2017-03-01   32.012036
2009-03-27    0.000000
2018-12-27   18.099139
2018-12-03   43.325629
2012-09-28   23.538185
2016-02-09   46.182045
2020-02-18   27.705211
2019-08-22   21.326631
2018-09-07   20.580350
2014-10-29   15.992378
2013-11-01   40.870889
2009-02-06    0.000000
2009-02-24    0.000000
2016-03-21   26.296663
2019-02-15   26.356619
2010-07-02   13.821799
2017-09-27   45.143934


In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# … your data prep up through df_cumulative …

start, end = "2020-01-02", "2020-03-30"
df_win = df_cumulative.loc[start:end]

# Make turbulence a Series, not DataFrame
turb_win = turb_per_date['turbulence'].loc[start:end]
turb_win.index = pd.to_datetime(turb_win.index)

fig, ax = plt.subplots(figsize=(15, 5))
df_win.plot(ax=ax, linewidth=1.5)
ax.set_xlim(start, end)

ax.xaxis.set_major_locator(mdates.DayLocator(interval=9))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.grid(axis='x', which='major', linestyle='--', linewidth=0.5)
ax.grid(axis='y', which='major', linestyle='--', linewidth=0.5)

ax.set_ylabel('Cumulative Return (%)')
ax.legend(['Ensemble', 'PPO', 'A2C', 'DJIA', 'Min var'])

# Twin axis — now turb_win is a Series
ax2 = ax.twinx()
ax2.plot(turb_win.index, turb_win.values,
         color='red', linestyle='--', linewidth=2,
         label='Turbulence Index')
ax2.set_ylabel('Turbulence Index')

# This will now be a single float
ax2.set_ylim(0, turb_win.max() * 1.1)

# Merge legends
lines, labs = ax.get_legend_handles_labels()
l2, la2 = ax2.get_legend_handles_labels()
ax.legend(lines + l2, labs + la2, loc='lower left', frameon=False)

plt.tight_layout()
plt.show()
